# Visualize
Show predictions on LIGYSIS of model finetuned on scPBD. 

In [ ]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

Gtk-Message: 15:31:52.957: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/E-regular-binding-site-predictor/evaluation/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/E-regular-binding-site-predictor/evaluation/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'
 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.
PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.


In [14]:
import sys
import numpy as np

DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_for_pocket_level_evaluation.csv'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/predictions'
PRECOMPUTED = False
AUTH = True  # whether to use auth or mmcif numbering
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/D-visualize')
import vis_utils

sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/B-evaluate-cryptoshow')
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import eval_utils
import cryptoshow_utils

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/GMMspherical,smoothing,PCA,GMM,AgglomerativeClust-finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())

if not PRECOMPUTED:
    # map binding residues to auth labeling
    binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket, auth=AUTH) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}
    model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket, auth=AUTH) for (pocket, _) in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}

else:
    import pickle
    with open(f'{PREDICTIONS_PATH}/model_predictions.pkl', 'rb') as f:
        model_predictions = pickle.load(f)
    with open(f'{PREDICTIONS_PATH}/binding_residues.pkl', 'rb') as f:
        binding_residues = pickle.load(f)


In [15]:
import pickle

with open(f'{PREDICTIONS_PATH}/GMMspherical,smoothing,PCA,GMM,AgglomerativeClust,model_predictions.pkl', 'wb') as f:
    pickle.dump(model_predictions, f)
with open(f'{PREDICTIONS_PATH}/GMMspherical,smoothing,PCA,GMM,AgglomerativeClust,binding_residues.pkl', 'wb') as f:
    pickle.dump(binding_residues, f)

In [33]:
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'

skip = True
for protein_id in binding_residues.keys():
    if protein_id != "1e9tA" and skip:
        continue
    skip = False
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.set('cif_use_auth', 0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    
    if len(this_model_predictions) != 0:
        cmd.color('forest', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
        intersection1 = vis_utils.get_intersection(this_binding_residues, this_model_predictions)
        if len(intersection1) != 0:
            cmd.color('yellow', vis_utils.generate_pymol_algebra_selection(protein_id, intersection1))


    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], [i for (i, _) in model_predictions_mmcifed[protein_id]], COORDINATES_DIR)
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')
    
    vis_utils.draw_pocket_centers(binding_residues_mmcifed[protein_id], [i for (i, _) in model_predictions_mmcifed[protein_id]], protein_id, COORDINATES_DIR)
    vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], [i for (i, _) in model_predictions_mmcifed[protein_id]], protein_id)

    cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

In [7]:
COLORS = ['pink', 'red', 'blue', 'green', 'brown', 'forest', 'sand', 'skyblue', 'slate', 'smudge', 'splitpea', 'sulfur', 'teal', 'tv_blue', 'tv_green', 'tv_orange', 'tv_red', 'tv_yellow']

COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'

for protein_id in binding_residues.keys():
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    print(f'Visualizing predictions for {protein_id}, {model_predictions[protein_id]} predicted pockets.')
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    

    for i, pocket in enumerate(model_predictions[protein_id]):
        cmd.color(COLORS[i % len(COLORS)], vis_utils.generate_pymol_algebra_selection(protein_id, pocket))

    cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

Visualizing predictions for 1a52A, [[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 88, 90, 91, 108, 229, 232], [54, 57, 58, 88, 90, 91, 92, 93, 94, 95, 98, 107, 108, 132, 221, 225], [92, 95, 125, 128, 129, 132, 221, 224, 225, 228], [47, 51, 88, 92, 125, 128, 129, 132, 221, 224, 225, 226, 227, 228, 229, 230, 231, 232]] predicted pockets.
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
Visualizing predictions for 1a5hD, [[47, 93, 183, 197, 198, 199, 200, 201, 202, 203, 221, 222, 223, 224, 225, 226, 227, 228, 229, 233, 234, 235, 236]] predicted pockets.
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF


In [2]:
import sys
import numpy as np

DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_for_pocket_level_evaluation.csv'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/predictions'
PRECOMPUTED = False
AUTH = True  # whether to use auth or mmcif numbering
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/D-visualize')
import vis_utils

sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/B-evaluate-cryptoshow')
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import eval_utils
import cryptoshow_utils

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/dbscan,eps=6,min_samples=[3,1],finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())

import pickle

with open('/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    mapping_df = pickle.load(file)

auth_to_label_chain_mapping = {}
for pdb_id, auth_chain_id, label_chain_id in zip(mapping_df['pdb_id'], mapping_df['auth_asym_id'], mapping_df['struct_asym_id']):
    auth_to_label_chain_mapping[(pdb_id, auth_chain_id)] = label_chain_id

In [4]:
COLORS = ['pink', 'red', 'blue', 'green', 'brown', 'forest', 'sand', 'skyblue', 'slate', 'smudge', 'splitpea', 'sulfur', 'teal', 'tv_blue', 'tv_green', 'tv_orange', 'tv_red', 'tv_yellow']

COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'

for protein_id in binding_residues_mmcifed.keys():
    label_chain_id = auth_to_label_chain_mapping[(protein_id[:4], protein_id[4:])]
    binding_residues = [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], label_chain_id, pocket, auth=False) for pocket in binding_residues_mmcifed[protein_id]]
    model_predictions = [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], label_chain_id, pocket, auth=False) for (pocket, _) in model_predictions_mmcifed[protein_id]]

    protein_id = f'{protein_id[:4]}{label_chain_id}'

    concatenated_binding_residues = np.concatenate(binding_residues) if binding_residues else np.array([])
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.set('cif_use_auth', 0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)

    for i, pocket in enumerate(model_predictions):
        cmd.color(COLORS[i % len(COLORS)], vis_utils.generate_pymol_algebra_selection(protein_id, pocket))

    cmd.show('surface', vis_utils.generate_pymol_algebra_selection(protein_id, concatenated_binding_residues))
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-